In [ ]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import functions
import math

In [ ]:
filepath ='/data/' #insert your correct filepath here!
csvfilenames = ['confinement_training.csv', 
                'member_information.csv', 
                'labresults_training.csv', 
                'medical_training.csv', 
                'rx_training.csv',
                'medical_target.csv',
                'rx_target.csv']

with open(filepath+csvfilenames[1]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

members = pd.DataFrame(data)
members.columns = members.iloc[0]
members = members[1:]
members = members.set_index(['PATID']) #makes PATID the row index instead of integers, not sure it's worth it.

with open(filepath+csvfilenames[2]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

labdata = pd.DataFrame(data)
labdata.columns = labdata.iloc[0]
labdata = labdata[1:]

with open(filepath+csvfilenames[3]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

allmedical = pd.DataFrame(data)
allmedical.columns = allmedical.iloc[0]
allmedical = allmedical[1:]

with open(filepath+csvfilenames[4]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

rxdata = pd.DataFrame(data)
rxdata.columns = rxdata.iloc[0]
rxdata = rxdata[1:]

with open(filepath+csvfilenames[5]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

targetallmedical = pd.DataFrame(data)
targetallmedical.columns = targetallmedical.iloc[0]
targetallmedical = targetallmedical[1:]

with open(filepath+csvfilenames[6]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

targetrxdata = pd.DataFrame(data)
targetrxdata.columns = targetrxdata.iloc[0]
targetrxdata = targetrxdata[1:]

In [ ]:
#Gets rid of pregnancy and emergency visits

medical_wanted = allmedical[('630' >  allmedical.DIAG1) | (allmedical.DIAG1 > '67999')]
medical_wanted = medical_wanted[('630' >  medical_wanted.DIAG2) | (medical_wanted.DIAG2 > '67999')]

medical_wanted = medical_wanted[('800' >  medical_wanted.DIAG1) | (medical_wanted.DIAG1 > '95999')]
medical_wanted = medical_wanted[('800' >  medical_wanted.DIAG2) | (medical_wanted.DIAG2 > '95999')]

medical_wanted = medical_wanted[medical_wanted['DIAG1'].str.startswith('E') == False]
medical_wanted = medical_wanted[medical_wanted['DIAG2'].str.startswith('E') == False]

medical_wanted = medical_wanted[(medical_wanted['DIAG1'].str.startswith('V3') == False) & (medical_wanted['DIAG1'].str.startswith('V22') == False) & (medical_wanted['DIAG1'].str.startswith('V23') == False) & (medical_wanted['DIAG1'].str.startswith('V24') == False) & (medical_wanted['DIAG1'].str.startswith('V28') == False) & (medical_wanted['DIAG1'].str.startswith('V91') == False)]
medical_wanted = medical_wanted[(medical_wanted['DIAG2'].str.startswith('V3') == False) & (medical_wanted['DIAG2'].str.startswith('V22') == False) & (medical_wanted['DIAG2'].str.startswith('V23') == False) & (medical_wanted['DIAG2'].str.startswith('V24') == False) & (medical_wanted['DIAG2'].str.startswith('V28') == False) & (medical_wanted['DIAG2'].str.startswith('V91') == False)]



medical_wanted_t = targetallmedical[('630' >  targetallmedical.DIAG1) | (targetallmedical.DIAG1 > '67999')]
medical_wanted_t = medical_wanted_t[('630' >  medical_wanted_t.DIAG2) | (medical_wanted_t.DIAG2 > '67999')]

medical_wanted_t = medical_wanted_t[('800' >  medical_wanted_t.DIAG1) | (medical_wanted_t.DIAG1 > '95999')]
medical_wanted_t = medical_wanted_t[('800' >  medical_wanted_t.DIAG2) | (medical_wanted_t.DIAG2 > '95999')]

medical_wanted_t = medical_wanted_t[medical_wanted_t['DIAG1'].str.startswith('E') == False]
medical_wanted_t = medical_wanted_t[medical_wanted_t['DIAG2'].str.startswith('E') == False]

medical_wanted_t = medical_wanted_t[(medical_wanted_t['DIAG1'].str.startswith('V3') == False) & 
                                    (medical_wanted_t['DIAG1'].str.startswith('V22') == False) & 
                                    (medical_wanted_t['DIAG1'].str.startswith('V23') == False) & 
                                    (medical_wanted_t['DIAG1'].str.startswith('V24') == False) & 
                                    (medical_wanted_t['DIAG1'].str.startswith('V28') == False) & 
                                    (medical_wanted_t['DIAG1'].str.startswith('V91') == False)]
medical_wanted_t = medical_wanted_t[(medical_wanted_t['DIAG2'].str.startswith('V3') == False) & 
                                    (medical_wanted_t['DIAG2'].str.startswith('V22') == False) & 
                                    (medical_wanted_t['DIAG2'].str.startswith('V23') == False) & 
                                    (medical_wanted_t['DIAG2'].str.startswith('V24') == False) & 
                                    (medical_wanted_t['DIAG2'].str.startswith('V28') == False) & 
                                    (medical_wanted_t['DIAG2'].str.startswith('V91') == False)]

In [ ]:
#Saves the wanted medical data to your current directory. NOTE! This might not be where you actually want to save them.
medical_wanted.to_csv('trainingmedicalwanted.csv')
medical_wanted_t.to_csv('targetmedicalwanted.csv')

In [ ]:
#Change everything to numeric data. This guy takes a while
medical_wanted['STD_COST']=medical_wanted['STD_COST'].apply(pd.to_numeric) #change entries to numbers
rxdata.loc[rxdata['STD_COST']=='None', 'STD_COST']=0 #change all nones to zeros
rxdata['STD_COST']=rxdata['STD_COST'].apply(pd.to_numeric) #change all entries to numbers
members['year_of_birth']=members['year_of_birth'].apply(pd.to_numeric)
rxdata['DAYS_FROM_DIAG'] = rxdata['DAYS_FROM_DIAG'].apply(pd.to_numeric)
medical_wanted['DAYS_FROM_DIAG'] = medical_wanted['DAYS_FROM_DIAG'].apply(pd.to_numeric)

medical_wanted_t['STD_COST']=medical_wanted_t['STD_COST'].apply(pd.to_numeric) #change entries to numbers
targetrxdata.loc[targetrxdata['STD_COST']=='None', 'STD_COST']=0 #change all nones to zeros
targetrxdata['STD_COST']=targetrxdata['STD_COST'].apply(pd.to_numeric) #change all entries to numbers
targetrxdata['DAYS_FROM_DIAG'] = targetrxdata['DAYS_FROM_DIAG'].apply(pd.to_numeric)
medical_wanted_t['DAYS_FROM_DIAG'] = medical_wanted_t['DAYS_FROM_DIAG'].apply(pd.to_numeric)

In [ ]:
#add total cost, total prescription cost and total medical cost to members data frame
members['totalcost'] = 0.0
members['rxcost'] = 0.0
members['medicalcost'] = 0.0

dummy = rxdata.groupby('PATID')['STD_COST'].sum()

for k in range(len(dummy)):
    members.at[dummy.index[k],'totalcost'] += dummy.iat[k]
    members.at[dummy.index[k], 'rxcost'] = dummy.iat[k]
    
dummy = medical_wanted.groupby('PATID')['STD_COST'].sum()

for k in range(len(dummy)):
    members.at[dummy.index[k], 'totalcost'] += dummy.iat[k]
    members.at[dummy.index[k], 'medicalcost'] = dummy.iat[k]
    

#add target data
members['targetcost'] = 0.0 
members['targetrxcost'] = 0.0
members['targetmedicalcost'] = 0.0

dummy = targetrxdata.groupby('PATID')['STD_COST'].sum()


for k in range(len(dummy)):
    members.at[dummy.index[k],'targetcost'] += dummy.iat[k]
    members.at[dummy.index[k], 'targetrxcost'] = dummy.iat[k]

    
dummy = medical_wanted_t.groupby('PATID')['STD_COST'].sum()


for k in range(len(dummy)):
    members.at[dummy.index[k], 'targetcost'] += dummy.iat[k]
    members.at[dummy.index[k], 'targetmedicalcost'] = dummy.iat[k]

In [ ]:
#For each patient, computes the first and last time you had a medical claim and the first and last time you
#filled a prescription
dummy = rxdata.groupby('PATID')['DAYS_FROM_DIAG'].min()
for k in range(len(dummy)):
    members.at[dummy.index[k], 'minrxday'] = dummy.iat[k]
dummy = rxdata.groupby('PATID')['DAYS_FROM_DIAG'].max()
for k in range(len(dummy)):
    members.at[dummy.index[k], 'maxrxday'] = dummy.iat[k]
dummy = medical_wanted.groupby('PATID')['DAYS_FROM_DIAG'].min()
for k in range(len(dummy)):
    members.at[dummy.index[k], 'minmedicalday'] = dummy.iat[k]
dummy = medical_wanted.groupby('PATID')['DAYS_FROM_DIAG'].max()
for k in range(len(dummy)):
    members.at[dummy.index[k], 'maxmedicalday'] = dummy.iat[k]

In [ ]:
#Split costs into prediagnosis and postdiagnosis
members['prediag_medicalcost'] = 0.0
members['postdiag_medicalcost'] = 0.0
members['prediag_rxcost'] = 0.0
members['postdiag_rxcost'] = 0.0
members['prediag_totalcost'] = 0.0
members['postdiag_totalcost'] = 0.0

prediag_med = medical_wanted[medical_wanted.DAYS_FROM_DIAG>=0].groupby('PATID')['STD_COST'].sum()
postdiag_med = medical_wanted[medical_wanted.DAYS_FROM_DIAG<0].groupby('PATID')['STD_COST'].sum()

prediag_rx = rxdata[rxdata.DAYS_FROM_DIAG>=0].groupby('PATID')['STD_COST'].sum()
postdiag_rx = rxdata[rxdata.DAYS_FROM_DIAG<0].groupby('PATID')['STD_COST'].sum()

for k in range(len(prediag_med)):
    members.at[prediag_med.index[k], 'prediag_medicalcost'] = prediag_med.iat[k]
    members.at[prediag_med.index[k], 'prediag_totalcost'] += prediag_med.iat[k]
for k in range(len(postdiag_med)):
    members.at[postdiag_med.index[k], 'postdiag_medicalcost'] = postdiag_med.iat[k]
    members.at[postdiag_med.index[k], 'postdiag_totalcost'] += postdiag_med.iat[k]
for k in range(len(prediag_rx)):
    members.at[prediag_rx.index[k], 'prediag_rxcost'] = prediag_rx.iat[k]
    members.at[prediag_rx.index[k], 'prediag_totalcost'] += prediag_rx.iat[k]
for k in range(len(postdiag_rx)):
    members.at[postdiag_rx.index[k], 'postdiag_rxcost'] = postdiag_rx.iat[k]
    members.at[postdiag_rx.index[k], 'postdiag_totalcost'] += postdiag_rx.iat[k]

In [ ]:
#Calculate average daily costs
members['prediag_days'] = 0       # First need to calculate number of days before and after diagnosis
members['postdiag_days'] = 0

members['prediag_days'] = -1 * members[['minmedicalday', 'minrxday']].min(axis=1) + 1
members['postdiag_days'] = members[['maxmedicalday', 'maxrxday']].max(axis=1) + 1

members['prediag_dailymedcost'] = members['prediag_medicalcost'] / members['prediag_days']
members['postdiag_dailymedcost'] = members['postdiag_medicalcost'] / members['postdiag_days']
members['prediag_dailyrxcost'] = members['prediag_rxcost'] / members['prediag_days']
members['postdiag_dailyrxcost'] = members['postdiag_rxcost'] / members['postdiag_days']
members['prediag_dailytotalcost'] = members['prediag_totalcost'] / members['prediag_days']
members['postdiag_dailytotalcost'] = members['postdiag_totalcost'] / members['postdiag_days']


members['dailymedicalcost'] = members['medicalcost'] / (members['prediag_days'] + members['postdiag_days'])
members['dailyrxcost'] = members['rxcost'] / (members['prediag_days'] + members['postdiag_days'])
members['dailytotalcost'] = members['totalcost'] / (members['prediag_days'] + members['postdiag_days'])

In [ ]:
#Compute the number of emergency visits for each patient
#emergency 'point of sale' codes are:
#emergency=['23','24', '21', '41', '20', '42']
medical_wanted_emergency = medical_wanted[(medical_wanted['POS'] =='23') | 
                                          (medical_wanted['POS'] =='24') | 
                                          (medical_wanted['POS'] =='21') |
                                          (medical_wanted['POS'] =='41') |
                                          (medical_wanted['POS'] =='20') |
                                          (medical_wanted['POS'] =='42') ]
members['emervisits']=0
patids = list(members.index)
#tick = time.clock() #uncomment the tick and tock if you want to know how much time this takes
patidgroups = medical_wanted_emergency.groupby('PATID')
for k in patids:
    try:
        members.loc[k, 'emervisits'] = len(patidgroups.get_group(k).groupby('DAYS_FROM_DIAG'))
    except KeyError:
        pass
#tock = time.clock()
#print tock-tick

In [ ]:
#Compute the number of times each patient saw a medical provider for psychiatric issues - inpatient and outpatient
#psychiatric=['53', '55', '51', '52', '57'] # 51 is inpatient faciliity, 55 is res. sub. abuse
medical_wanted_psych = medical_wanted[(medical_wanted['POS'] =='53') | 
                                          #(medical_wanted['POS'] =='55') |
                                          (medical_wanted['POS'] =='52') |
                                          (medical_wanted['POS'] =='57')]
members['psychvisits']=0
patids = list(members.index)
#tick = time.clock()
patidgroups = medical_wanted_psych.groupby('PATID')
for k in patids:
    try:
        members.loc[k, 'psychvisits'] = len(patidgroups.get_group(k).groupby('DAYS_FROM_DIAG'))
    except KeyError:
        pass
#tock = time.clock()
#print tock-tick

In [ ]:
#Compute the number of 'regular' doctors office visits for each patient
#regular=['11', '01', '81','22', '60', '49', '71', '62', '17', '50', '61', '03']
medical_wanted_office = medical_wanted[(medical_wanted['POS'] =='11') | 
                                          (medical_wanted['POS'] =='01') | 
                                          (medical_wanted['POS'] =='81') |
                                          (medical_wanted['POS'] =='22') |
                                          (medical_wanted['POS'] =='60') |
                                      (medical_wanted['POS'] =='49') | 
                                          (medical_wanted['POS'] =='71') |
                                          (medical_wanted['POS'] =='62') |
                                          (medical_wanted['POS'] =='17') |
                                      (medical_wanted['POS'] =='50') | 
                                          (medical_wanted['POS'] =='61') |
                                          (medical_wanted['POS'] =='03') ]
members['officevisits']=0
patids = list(members.index)
#tick = time.clock()
patidgroups = medical_wanted_office.groupby('PATID')
for k in patids:
    try:
        members.loc[k, 'officevisits'] = len(patidgroups.get_group(k).groupby('DAYS_FROM_DIAG'))
    except KeyError:
        pass
#tock = time.clock()
#print tock-tick

In [ ]:
#Compute the number of claims for elderly care for each patient
#elderly=['13', '31', '32', '33']
medical_wanted_elderly = medical_wanted[(medical_wanted['POS'] =='13') | 
                                          (medical_wanted['POS'] =='31') | 
                                          (medical_wanted['POS'] =='32') |
                                          (medical_wanted['POS'] =='33') ]
members['elderlyvisits']=0
patids = list(members.index)
#tick = time.clock()
patidgroups = medical_wanted_elderly.groupby('PATID')
for k in patids:
    try:
        members.loc[k, 'elderlyvisits'] = len(patidgroups.get_group(k).groupby('DAYS_FROM_DIAG'))
    except KeyError:
        pass
#tock = time.clock()
#print tock-tick

In [ ]:
#Compute the number of home health care instances for each patient
#home=['12', '14', '15']
medical_wanted_home = medical_wanted[(medical_wanted['POS'] =='12') | 
                                          (medical_wanted['POS'] =='14') | 
                                          (medical_wanted['POS'] =='15')]
members['homevisits']=0
patids = list(members.index)
#tick = time.clock()
patidgroups = medical_wanted_home.groupby('PATID')
for k in patids:
    try:
        members.loc[k, 'homevisits'] = len(patidgroups.get_group(k).groupby('DAYS_FROM_DIAG'))
    except KeyError:
        pass
#tock = time.clock()
#print tock-tick

In [ ]:
#Compute the number of dialysis claims for each patient
#dialysis=['65']
medical_wanted_dialysis = medical_wanted[(medical_wanted['POS'] =='65')]
members['dialysisvisits']=0
patids = list(members.index)
#tick = time.clock()
patidgroups = medical_wanted_dialysis.groupby('PATID')
for k in patids:
    try:
        members.loc[k, 'dialysisvisits'] = len(patidgroups.get_group(k).groupby('DAYS_FROM_DIAG'))
    except KeyError:
        pass
#tock = time.clock()
#print tock-tick

In [ ]:
#Compute the number of hospice claims for each patient
#hospice=['34']
medical_wanted_hospice = medical_wanted[(medical_wanted['POS'] =='34')]
members['hospicevisits']=0
patids = list(members.index)
#tick = time.clock()
patidgroups = medical_wanted_hospice.groupby('PATID')
for k in patids:
    try:
        members.loc[k, 'hospicevisits'] = len(patidgroups.get_group(k).groupby('DAYS_FROM_DIAG'))
    except KeyError:
        pass
#tock = time.clock()
#print tock-tick

In [ ]:
#Number of medical claims of unknown origin for each patient
#unknown=['99', 'None']
medical_wanted_unknown = medical_wanted[(medical_wanted['POS'] =='99')|
                                        (medical_wanted['POS'] == 'None')]
members['unknownvisits']=0
patids = list(members.index)
#tick = time.clock()
patidgroups = medical_wanted_unknown.groupby('PATID')
for k in patids:
    try:
        members.loc[k, 'unknownvisits'] = len(patidgroups.get_group(k).groupby('DAYS_FROM_DIAG'))
    except KeyError:
        pass
#tock = time.clock()
#print tock-tick

In [ ]:
#Add the geographic region of each member
#Midwest, Northeast, South, West, Unknown
members['region'] = members.apply(lambda row: functions.US_region(row), axis=1)

#uncomment the following if you want the plot of average total cost per region
#region_series = members.groupby('region')['totalcost'].mean()  

#region_series.plot.bar()
#plt.show()

In [ ]:
#we may need to make more things numeric than already are

In [ ]:
#Make binary variables for whether a patient has ever had a specific diagnosis
members['infpardisdiag']=0 #001-139
members['neoplasmsdiag']=0 #140-239
#members['endocrinediag']=1 #240-279, needs to be split up
members['blooddiag']=0 #280-289
members['mentaldiag']=0 #290-319
members['nervediag']=0 #320-359
members['sensediag']=0 #360-389
members['circdiag']=0 #390-489
members['respdiag']=0 #460-519
members['digestdiag']=0 #520-579
members['genitdiag']=0 #580-629
members['skindiag']=0 #608-709
members['muscskeldiag']=0 #710-739
members['conganomdiag']=0 #740-759
members['illdefdiag']=0 #780-799

dummy = medical_wanted.groupby('PATID')
patids = list(members.index)
for k in patids:
    try:
        tinydummy = dummy.get_group(k)
        if len(tinydummy[(tinydummy['DIAG1']>='001') & (tinydummy['DIAG1']<='13999')]):
            members.loc[k,'infpardisdiag']=1
        if len(tinydummy[(tinydummy['DIAG1']>='140') & (tinydummy['DIAG1']<='23999')]):
            members.loc[k,'neoplasmsdiag']=1
        if len(tinydummy[(tinydummy['DIAG1']>='280') & (tinydummy['DIAG1']<='28999')]):
            members.loc[k,'blooddiag']=1
        if len(tinydummy[(tinydummy['DIAG1']>='290') & (tinydummy['DIAG1']<='31999')]):
            members.loc[k,'mentaldiag']=1
        if len(tinydummy[(tinydummy['DIAG1']>='320') & (tinydummy['DIAG1']<='35999')]):
            members.loc[k,'nervediag']=1
        if len(tinydummy[(tinydummy['DIAG1']>='360') & (tinydummy['DIAG1']<='38999')]):
            members.loc[k,'sensediag']=1
        if len(tinydummy[(tinydummy['DIAG1']>='390') & (tinydummy['DIAG1']<='48999')]):
            members.loc[k,'circdiag']=1
        if len(tinydummy[(tinydummy['DIAG1']>='460') & (tinydummy['DIAG1']<='51999')]):
            members.loc[k,'respdiag']=1
        if len(tinydummy[(tinydummy['DIAG1']>='520') & (tinydummy['DIAG1']<='57999')]):
            members.loc[k,'digestdiag']=1
        if len(tinydummy[(tinydummy['DIAG1']>='580') & (tinydummy['DIAG1']<='62999')]):
            members.loc[k,'genitdiag']=1
        if len(tinydummy[(tinydummy['DIAG1']>='608') & (tinydummy['DIAG1']<='70999')]):
            members.loc[k,'skindiag']=1
        if len(tinydummy[(tinydummy['DIAG1']>='710') & (tinydummy['DIAG1']<='73999')]):
            members.loc[k,'muscskeldiag']=1
        if len(tinydummy[(tinydummy['DIAG1']>='740') & (tinydummy['DIAG1']<='75999')]):
            members.loc[k,'conganomdiag']=1
        if len(tinydummy[(tinydummy['DIAG1']>='780') & (tinydummy['DIAG1']<='79999')]):
            members.loc[k,'illdefdiag']=1  
    except KeyError:
        pass

In [ ]:
#Make binary variables for specific endocrine disorder diagnosis codes
#endocrine divisions
members['thyroiddiag']=0 #240-246

members['2diabmellnocomp']=0 #2490
members['2diabmellketoa']=0 #2491
members['2diabmellosmol']=0 #2492
members['2diabmellcoma']=0 #2493
members['2diabmellrenal']=0 #2494
members['2diabmellopth']=0 #2495
members['2diabmellneuro']=0 #2496
members['2diabmellcirc']=0 #2497
members['2diabmellother']=0 #2498
members['2diabmellunspec']=0 #2499

members['diabmellnocomp']=0 #2500
members['diabmellketoa']=0 #2501
members['diabmellosmol']=0 #2502
members['diabmellcoma']=0 #2503
members['diabmellrenal']=0 #2504
members['diabmellopth']=0 #2505
members['diabmellneuro']=0 #2506
members['diabmellcirc']=0 #2507
members['diabmellother']=0 #2508
members['diabmellunspec']=0 #2509

members['otherendodiag']=0 #251-259
members['nutritdefdiag']=0 #269-269
members['metabimmdiag']=0 #270-279

dummy = medical_wanted.groupby('PATID')
patids = list(members.index)
for k in patids:
    try:
        tinydummy = dummy.get_group(k)
        if len(tinydummy[(tinydummy['DIAG1']>='240') & (tinydummy['DIAG1']<='24699')]):
            members.loc[k,'thyroiddiag']=1
            
        if len(tinydummy[(tinydummy['DIAG1']>='2490') & (tinydummy['DIAG1']<='24909')]):
            members.loc[k,'2diabmellnocomp']=1
        if len(tinydummy[(tinydummy['DIAG1']>='2491') & (tinydummy['DIAG1']<='24919')]):
            members.loc[k,'2diabmellketoa']=1
        if len(tinydummy[(tinydummy['DIAG1']>='2492') & (tinydummy['DIAG1']<='24929')]):
            members.loc[k,'2diabmellosmol']=1    
        if len(tinydummy[(tinydummy['DIAG1']>='2493') & (tinydummy['DIAG1']<='24939')]):
            members.loc[k,'2diabmellcoma']=1
        if len(tinydummy[(tinydummy['DIAG1']>='2494') & (tinydummy['DIAG1']<='24949')]):
            members.loc[k,'2diabmellrenal']=1  
        if len(tinydummy[(tinydummy['DIAG1']>='2495') & (tinydummy['DIAG1']<='24959')]):
            members.loc[k,'2diabmellopth']=1
        if len(tinydummy[(tinydummy['DIAG1']>='2496') & (tinydummy['DIAG1']<='24969')]):
            members.loc[k,'2diabmellneuro']=1
        if len(tinydummy[(tinydummy['DIAG1']>='2497') & (tinydummy['DIAG1']<='24979')]):
            members.loc[k,'2diabmellcirc']=1
        if len(tinydummy[(tinydummy['DIAG1']>='2498') & (tinydummy['DIAG1']<='24989')]):
            members.loc[k,'2diabmellother']=1
        if len(tinydummy[(tinydummy['DIAG1']>='2499') & (tinydummy['DIAG1']<='24999')]):
            members.loc[k,'2diabmellunspec']=1
        
        if len(tinydummy[(tinydummy['DIAG1']>='2500') & (tinydummy['DIAG1']<='25009')]):
            members.loc[k,'diabmellnocomp']=1
        if len(tinydummy[(tinydummy['DIAG1']>='2501') & (tinydummy['DIAG1']<='25019')]):
            members.loc[k,'diabmellketoa']=1
        if len(tinydummy[(tinydummy['DIAG1']>='2502') & (tinydummy['DIAG1']<='25029')]):
            members.loc[k,'diabmellosmol']=1    
        if len(tinydummy[(tinydummy['DIAG1']>='2503') & (tinydummy['DIAG1']<='25039')]):
            members.loc[k,'diabmellcoma']=1
        if len(tinydummy[(tinydummy['DIAG1']>='2504') & (tinydummy['DIAG1']<='25049')]):
            members.loc[k,'diabmellrenal']=1  
        if len(tinydummy[(tinydummy['DIAG1']>='2505') & (tinydummy['DIAG1']<='25059')]):
            members.loc[k,'diabmellopth']=1
        if len(tinydummy[(tinydummy['DIAG1']>='2506') & (tinydummy['DIAG1']<='25069')]):
            members.loc[k,'diabmellneuro']=1
        if len(tinydummy[(tinydummy['DIAG1']>='2507') & (tinydummy['DIAG1']<='25079')]):
            members.loc[k,'diabmellcirc']=1
        if len(tinydummy[(tinydummy['DIAG1']>='2508') & (tinydummy['DIAG1']<='25089')]):
            members.loc[k,'diabmellother']=1
        if len(tinydummy[(tinydummy['DIAG1']>='2509') & (tinydummy['DIAG1']<='25099')]):
            members.loc[k,'diabmellunspec']=1
            
        tinydummy = dummy.get_group(k)
        if len(tinydummy[(tinydummy['DIAG1']>='251') & (tinydummy['DIAG1']<='25999')]):
            members.loc[k,'otherendodiag']=1
        tinydummy = dummy.get_group(k)
        if len(tinydummy[(tinydummy['DIAG1']>='260') & (tinydummy['DIAG1']<='26999')]):
            members.loc[k,'nutridefdiag']=1
        tinydummy = dummy.get_group(k)
        if len(tinydummy[(tinydummy['DIAG1']>='270') & (tinydummy['DIAG1']<='27999')]):
            members.loc[k,'metabimmdiag']=1
    except KeyError:
        pass

In [ ]:
#make binary variables for whether a patient has ever had a certain type of prescription
#rxdata
members['allergies'] =0 # 04
members['antiinfectives']=0 # 08
members['antineoplastic']=0 #10
members['autonomic']=0 #12
members['bloodfromcoag']=0 #20
members['cardiovascular']=0 #24
members['centralnervous']=0 #28
members['contraceptives']=0 #32
members['dental'] = 0 #34
members['electrolytic']=0 #40
members['enzymes']=0 #44
members['coughstuff']=0 #48
members['opthamlic']=0 #52
members['gastrointestinal']=0 #56
members['hormones']=0 #68
members['oxytocics']=0 #76
members['skinstuff']=0 #84
members['vitamins']=0 #88
members['asthma/alcohol'] = 0 #92

dummy = rxdata.groupby('PATID')
patids = list(members.index)
for k in patids:
    try:
        tinydummy = dummy.get_group(k)
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('04'))]):
            members.loc[k,'allergies']=1
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('08'))]):
            members.loc[k,'antiinfectives']=0
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('10'))]):
            members.loc[k,'antineoplastic']=1
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('12'))]):
            members.loc[k,'autonomic']=1
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('20'))]):
            members.loc[k,'bloodfromcoag']=1
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('24'))]):
            members.loc[k,'cardiovascular']=1
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('28'))]):
            members.loc[k,'centralnervous']=1
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('32'))]):
            members.loc[k,'contraceptives']=1
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('34'))]):
            members.loc[k,'dental']=1
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('40'))]):
            members.loc[k,'electrolytic']=1
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('44'))]):
            members.loc[k,'enzymes']=1
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('48'))]):
            members.loc[k,'coughstuff']=1
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('52'))]):
            members.loc[k,'opthamlic']=1
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('56'))]):
            members.loc[k,'gastrointestinal']=1
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('68'))]):
            members.loc[k,'hormones']=1
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('76'))]):
            members.loc[k,'oxytocics']=1
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('84'))]):
            members.loc[k,'skinstuff']=1
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('88'))]):
            members.loc[k,'vitamins']=1
        if len(tinydummy[(tinydummy['AHFSCLSS'].str.startswith('92'))]):
            members.loc[k,'asthma/alcohol']=1
    except KeyError:
        pass

In [ ]:
#Find the first and last time a patient filled a prescription and the first and last time they visited
#a doctor
dummy = rxdata.groupby('PATID')['DAYS_FROM_DIAG'].min()
rx_patids = list(dummy.index)
for k in range(len(rx_patids)):
    members.at[rx_patids[k], 'minrxday'] = dummy.at[rx_patids[k]]
dummy = rxdata.groupby('PATID')['DAYS_FROM_DIAG'].max()
rx_patids = list(dummy.index)
for k in range(len(rx_patids)):
    members.at[rx_patids[k], 'maxrxday'] = dummy.at[rx_patids[k]]
dummy = medical_wanted.groupby('PATID')['DAYS_FROM_DIAG'].min()
med_patids = list(dummy.index)
for k in range(len(med_patids)):
    members.at[med_patids[k], 'minmedicalday'] = dummy.at[med_patids[k]]
dummy = medical_wanted.groupby('PATID')['DAYS_FROM_DIAG'].max()
med_patids = list(dummy.index)
for k in range(len(med_patids)):
    members.at[med_patids[k], 'maxmedicalday'] = dummy.at[med_patids[k]]

In [ ]:
#Find patients who never had prescriptions, never had a doctor's visit or both
no_rx=members[members['minrxday'].isnull()==True]
no_med=members[members['minmedicalday'].isnull()==True]
neither=members[(members['maxmedicalday'].isnull()==True) & (members['maxrxday'].isnull()==True)]

In [ ]:
#start correcting the variables
rxmaxbypatid=rxdata.groupby('PATID')['DAYS_FROM_DIAG'].max() #find max rx day for each patient
medlistbypatid = medical_wanted.groupby('PATID')['DAYS_FROM_DIAG'] #create list of all diagnosis days for each patient

#Find the day of the last medical visit before the last prescription fill
#still a bit glitchy, will fix tomorrow
medical_wanted["previousmedday"] = np.nan
key=list(set(medlistbypatid.groups.keys()).intersection(rxmaxbypatid.index))
for k in range(5):
    medical_wanted.loc[medical_wanted['PATID']==key[k],'previousmedday'] = max(filter(lambda x:x<=rxmaxbypatid.at[key[k]],medlistbypatid.get_group(key[k])))

In [ ]:
#run me last!!
members.to_csv('members2.csv')